In [7]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [8]:
#Get relative path of the CSV File 
#Enumerate therough all the files in the folder and check if the path exists

mypath = os.path.dirname(os.path.realpath('train.csv'))
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
os.path.exists(mypath)

True

In [9]:
print(onlyfiles[4])

train.csv


In [10]:
#Read Training CSV for house regression
dataset = pd.read_csv(onlyfiles[4])
dataset.shape

(1460, 81)

importing the dataset 
encode categorical data
Backward elimination

In [11]:
#Delete redundant features with over 50% NAN values E.G: porch sizes, pool area, alley, miscallenous features and its value 
unwantedFeaturesList = ["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea",
                       "PoolQC","Fence","Alley","MiscFeature","MiscVal"]
dataset.drop(unwantedFeaturesList,inplace=True,axis=1)

In [12]:
# Check all the columns fetures whcih has a numerical value type
is_number = np.vectorize(lambda x: np.issubdtype(x, np.number))
is_number(dataset.dtypes)

array([ True,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True, False,  True,
        True,  True, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True, False, False,  True, False,  True,  True, False,
       False, False,  True,  True, False, False,  True], dtype=bool)

In [109]:
#Separate and select dataframes by dataTypes 
df_NonNumericalType = dataset.select_dtypes(exclude=[np.number])
df_NumericalType = dataset.select_dtypes(include=[np.number])

#Set type to categorical
df_NonNumericalType[df_NonNumericalType.select_dtypes(['object']).columns] = df_NonNumericalType.select_dtypes(['object']).apply(lambda x: x.astype('category'))
df_NonNumericalType.dtypes

MSZoning         category
Street           category
LotShape         category
LandContour      category
Utilities        category
LotConfig        category
LandSlope        category
Neighborhood     category
Condition1       category
Condition2       category
BldgType         category
HouseStyle       category
RoofStyle        category
RoofMatl         category
Exterior1st      category
Exterior2nd      category
MasVnrType       category
ExterQual        category
ExterCond        category
Foundation       category
BsmtQual         category
BsmtCond         category
BsmtExposure     category
BsmtFinType1     category
BsmtFinType2     category
Heating          category
HeatingQC        category
CentralAir       category
Electrical       category
KitchenQual      category
Functional       category
FireplaceQu      category
GarageType       category
GarageFinish     category
GarageQual       category
GarageCond       category
PavedDrive       category
SaleType         category
SaleConditio

In [115]:
#Encoding categorical data
#Encoding the independent variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_NonNumericalType = LabelEncoder()
df_NonNumericalencoded.reset_index(level=0, inplace=True)
df_NumericalType.reset_index(level=0, inplace=True)

In [125]:
#Create dummy data for all the categorical columns
cat_features = [key for key in dict(df_NonNumericalType.dtypes) if dict(df_NonNumericalType.dtypes)[key] in ['category']]
df_NonNumericalencoded = pd.get_dummies(df_NonNumericalType, columns=cat_features, drop_first=True)
#pd.concat([df_NumericalType, df_NonNumericalencoded], ignore_index=False)



#Errors in implementing the one Hot encoder method
#df_NonNumericalType[:,:] = labelencoder_NonNumericalType.fit_transform(df_NonNumericalType[:,:])
#for features in range [cat_features]:
#    onehotencoder = OneHotEncoder(categorical_features = features )
#df_NonNumericalType = onehotencoder.fit_transform(df_NonNumericalType).toarray()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,...,SaleType_WD,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_NoSeWa,YearBuilt,YearRemodAdd,YrSold,index,level_0
0,856.0,854.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.0,856.0,NaN,2003.0,2003.0,2008.0,0.0,0.0
1,1262.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.0,1262.0,NaN,1976.0,1976.0,2007.0,1.0,1.0
2,920.0,866.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.0,920.0,NaN,2001.0,2002.0,2008.0,2.0,2.0
3,961.0,756.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,756.0,NaN,1915.0,1970.0,2006.0,3.0,3.0
4,1145.0,1053.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.0,1145.0,NaN,2000.0,2000.0,2008.0,4.0,4.0
5,796.0,566.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,796.0,NaN,1993.0,1995.0,2009.0,5.0,5.0
6,1694.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,1686.0,NaN,2004.0,2005.0,2007.0,6.0,6.0
7,1107.0,983.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,1107.0,NaN,1973.0,1973.0,2009.0,7.0,7.0
8,1022.0,752.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.0,952.0,NaN,1931.0,1950.0,2008.0,8.0,8.0
9,1077.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,991.0,NaN,1939.0,1950.0,2008.0,9.0,9.0


In [120]:
#x = dataset.iloc[:,:-1].values
#y = dataset.iloc[:,80].values